# Releasing LM-Evaluation-Harness v0.4.0

With the vast amount of work done in the field today, it helps to have a tool that people can use easily to share their results and use to check others to ensure reported numbers are valid. The LM Evaluation Harness is one such tool the community has used extensively. We want to continue to support the community and with that in mind, we’re excited to announce a major update on the LM Evaluation Harness to further our goal for open and accessible AI research.

Our refactor stems from our desires to make the following believed best practices easier to carry out.  

1.   Never copy results from other papers
2.   Always share your exact prompts
3.   Always provide model outputs
4.   Qualitatively review a small batch of outputs before running evaluation jobs at scale

We also wanted to make the library a better experience to use and to contribute or design evaluations within. New features in the new release that serve this purpose include:

1. Faster Evaluation Runtimes (accelerated data-parallel inference with HF Transformers + Accelerate, and commonly used or faster inference libraries such as vLLM and Llama-CPP)
2. Easier addition and sharing of new tasks (YAML-based task config formats, allowing single-file sharing of custom tasks)
3. More configurability, for more advanced workflows and easier operation with modifying prompts
4. Better logging of data at runtime and post-hoc

In this notebook we will be going through a short tutorial on how things work.

## Install LM-Eval

In [16]:
python -m pip install --upgrade pip

Selected Tasks: ['afrihate_amharic_hate']


2025-11-03:17:26:29 INFO     [__main__:348] Including path: ./
2025-11-03:17:36:01 WARNING  [__main__:369]  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
2025-11-03:17:36:03 INFO     [evaluator:202] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-11-03:17:36:03 INFO     [evaluator:240] Initializing hf model, with arguments: {'pretrained': 'EleutherAI/pythia-2.8b'}
2025-11-03:17:36:03 INFO     [models.huggingface:155] Device not specified
2025-11-03:17:36:03 INFO     [models.huggingface:156] Cuda Available? False
2025-11-03:17:36:06 DEBUG    [models.huggingface:528] Using model type 'causal'
2025-11-03:17:36:13 INFO     [models.huggingface:414] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

In [ ]:
pip install -q "git+https://github.com/EleutherAI/lm-evaluation-harness.git" --no-cache-dir

: 

## Create new evaluation tasks with config-based tasks

Even within the same task, many works have reported numbers based on different choices of evaluation. Some report on the test sets, validation sets, or even subset of the training sets. Others have specialized prompts and verbalizers. We introduce YAMLs to allow users to easily make different variations. By leveraging the YAML configs to configure evaluations, the refactored LM-Eval takes the methods of the `Task` object and makes them configurable by setting the appropriate attributes in the config file. There, users can set the tasks they want by setting the name of the HF dataset (local tasks are also possible), the dataset splits used, and much more. Key configurations relating to prompting, such as `doc_to_text`, previously implemented as a method of the same name, are now configurable with jinja2 to allow high-level scripting to transform a HF dataset to text string as input to the model.



A core-feature to LM-Eval is to configure tasks with YAML configs. With configs, you can fill preset fields to easily set up a task.

Here, we write a demo YAML config for a multiple-choice evaluation of BoolQ:

In [ ]:
YAML_amharic_hate_string = """
task: afrihate_amharic_hate
dataset_path: afrihate/afrihate
dataset_name: amh
output_type: multiple_choice
training_split: train
validation_split: validation 
test_split: test
fewshot_split: dev
fewshot_config:
  sampler: first_n
doc_to_text: "Tweet: {{text}}\\nQuestion: Is this tweet abusive, hateful, or normal?\\nAnswer:"
doc_to_target: label
doc_to_choice: ["abusive", "hate", "normal"]
should_decontaminate: true
doc_to_decontamination_query: text
metric_list:
  - metric: acc
  - metric: f1_macro
"""

with open("afrihate_amharic.yaml", "w") as f:
    f.write(YAML_amharic_hate_string)

update lm Harness

In [ ]:
pip install -U lm-eval

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\johnw\AppData\Local\Programs\Python\Python313\Scripts\pip.exe\__main__.py", line 4, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip'


And we can now run evaluation on this task, by pointing to the config file we've just created:

In [ ]:
%env LOGLEVEL=DEBUG
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks afrihate \
    --device cuda:0 \
    --batch_size auto \
    --limit 10

env: LOGLEVEL=DEBUG
^C


In [ ]:
!pip install -q "git+https://github.com/EleutherAI/lm-evaluation-harness.git" --no-cache-dir

Often, tasks are part of a larger group used to measure different capabilities. The dynamism of the field today means new dimensions of evaluation can come about which would mix and match new and older tasks alike. In LM-Eval, We can also group tasks and call that the group name to evaluate on a set of tasks easily. In this instance, let's evaluate the tag `yes_or_no_tasks` which comprise of the tasks `demo_boolq` and `demo_cola`; tasks which are multiple choice tasks with options `yes` and `no` as the name suggests.

<!-- making new groups is easier than ever, allowing user to work bottom-up by makiing individual tasks and linking them to a group or Top-Down, making a new group by listing existing tasks.

We also show the aggregate across samples besides only showing the aggregation between subtasks. This may come in handy when certain groups want to be aggregated as a single task. -->




In [ ]:
YAML_cola_string = """
tag: yes_or_no_tasks 
task: demo_cola
dataset_path: glue
dataset_name: cola
output_type: multiple_choice
training_split: train
validation_split: validation
doc_to_text: "{{sentence}}\nQuestion: Does this sentence make sense?\nAnswer:"
doc_to_target: label
doc_to_choice: ["no", "yes"]
should_decontaminate: true
doc_to_decontamination_query: sentence
metric_list:
  - metric: acc
"""
with open("cola.yaml", "w") as f:
    f.write(YAML_cola_string)

In [ ]:
# !accelerate launch --no_python
%env LOGLEVEL=DEBUG
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks yes_or_no_tasks \
    --limit 10 \
    --output output/yes_or_no_tasks/ \
    --log_samples

## Edit Prompt Templates Quickly

The following is a yaml made to evaluate the specific subtask of `high_school_geography` from MMLU. It uses the standard prompt where the we choose the letters from the options with most likelihood as the model's prediction.

In [ ]:
YAML_mmlu_geo_string = """
task: demo_mmlu_high_school_geography
dataset_path: cais/mmlu
dataset_name: high_school_geography
description: "The following are multiple choice questions (with answers) about high school geography.\n\n"
test_split: test
fewshot_split: dev
fewshot_config:
  sampler: first_n
output_type: multiple_choice
doc_to_text: "{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:"
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: answer
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
"""
with open("mmlu_high_school_geography.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

In [ ]:
# !accelerate launch --no_python
%env LOGLEVEL=DEBUG
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography \
    --limit 10 \
    --output output/mmlu_high_school_geography/ \
    --log_samples

We could also evaluate this task in a different way. For example, instead of observing the loglikelihood of the letters, we can instead evaluate on the choices themselves as the continuation. This is done by simply changing `doc_to_choice` from a list of letters to the corresponding `choices` field from the HF dataset. We write `"{{choices}}"` so that the string field is interpreted as jinja string that acquires the list from the HF dataset directly.

Another convenient feature here is since we're only modifying the `doc_to_choice` and the rest of config is the same as the task above, we can use the above configuration as a template by using `include: mmlu_high_school_geography.yaml` to load the config from that file. We'll need to add a unique task name as to not colide with the existing yaml config we're including. For this case we'll simply name this one `mmlu_high_school_geography_continuation`. `doc_to_text` is added here just for sake of clarity.

In [ ]:
YAML_mmlu_geo_string = """
include: mmlu_high_school_geography.yaml
task: demo_mmlu_high_school_geography_continuation
doc_to_text: "{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:"
doc_to_choice: "{{choices}}"
"""
with open("mmlu_high_school_geography_continuation.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

In [ ]:
# !accelerate launch --no_python
%env LOGLEVEL=DEBUG
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography_continuation \
    --limit 10 \
    --output output/mmlu_high_school_geography_continuation/ \
    --log_samples

If we take a look at the samples, we can see that it is in fact evaluating the continuation based on the choices rather than the letters.

In [ ]:
from google.colab import files


files.view(
    "output/mmlu_high_school_geography_continuation/pretrained__EleutherAI__pythia-2.8b_demo_mmlu_high_school_geography_continuation.jsonl"
)

## Closer Look at YAML Fields

To prepare a task we can simply fill in a YAML config with the relevant information.

`output_type`
The current provided evaluation types comprise of the following:
1.   `loglikelihood`: Evaluates the loglikelihood of a continuation, conditioned on some input string.
2.   `loglikelihood_rolling`: evaluate the loglikelihood of producing a string, conditioned on the empty string. (Used for perplexity evaluations)
3.   `multiple_choice`: Evaluates loglikelihood among the a number of choices predicted by the model.
4.   `greedy_until`: Model outputs greedy generation (can be configured to to use beam search and other generation-related parameters)

The core prompt revolves around 3 fields.
1. `doc_to_text`: Denotes the prompt template that will be used as input to the model.
2. `doc_to_choice`: Available choices that will be used as continuation for the model. This is used when the `output_type` is `multiple_choice`, and otherwise can be left as `None`.
3. `doc_to_target`: When `output_type` is `multiple_choice`, this can be an index that corresponds to the correct answer, or the answer string itself (must be a subset of `doc_to_choice`). For other tasks, this is expected to be a string. You can fill this field with a feature name from the HF dataset so long as the resulting feature follows the conditioned described.

These three fields can be expressed as strings, column names from the source dataset, or as Jinja2 templates that can use fields from the source dataset as variables.


## What if Jinja is not Sufficient?

There can be times where the Jinja2 templating language is not enough to make the prompt we had in mind. There are a few ways to circumvent this limitation:

1. Use `!function` operator for the prompt-related fields to pass a python function that takes as input the dataset row, and will output the prompt template component.
2. Perform a transformation on the dataset beforehand.

Below, we show an example of using `!function` to create `doc_to_text` from a python function:

In [ ]:
YAML_mmlu_geo_string = """
include: mmlu_high_school_geography.yaml
task: demo_mmlu_high_school_geography_function_prompt
doc_to_text: !function utils.doc_to_text
doc_to_choice: "{{choices}}"
"""
with open("demo_mmlu_high_school_geography_function_prompt.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

DOC_TO_TEXT = """
def doc_to_text(x):
    question = x["question"].strip()
    choices = x["choices"]
    option_a = choices[0]
    option_b = choices[1]
    option_c = choices[2]
    option_d = choices[3]
    return f"{question}\\nA. {option_a}\\nB. {option_b}\\nC. {option_c}\\nD. {option_d}\\nAnswer:"
"""
with open("utils.py", "w") as f:
    f.write(DOC_TO_TEXT)

!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography_function_prompt \
    --limit 10 \
    --output output/demo_mmlu_high_school_geography_function_prompt/ \
    --log_samples

Next, we'll also show how to do this via preprocessing the dataset as necessary using the `process_docs` config field:

We will write a function that will modify each document in our evaluation dataset's split to add a field that is suitable for us to use in `doc_to_text`.

In [ ]:
YAML_mmlu_geo_string = """
include: mmlu_high_school_geography.yaml
task: demo_mmlu_high_school_geography_function_prompt_2
process_docs: !function utils_process_docs.process_docs
doc_to_text: "{{input}}"
doc_to_choice: "{{choices}}"
"""
with open("demo_mmlu_high_school_geography_process_docs.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

DOC_TO_TEXT = """
def process_docs(dataset):
    def _process_doc(x):
        question = x["question"].strip()
        choices = x["choices"]
        option_a = choices[0]
        option_b = choices[1]
        option_c = choices[2]
        option_d = choices[3]
        doc["input"] = f"{question}\\nA. {option_a}\\nB. {option_b}\\nC. {option_c}\\nD. {option_d}\\nAnswer:"
        return out_doc

    return dataset.map(_process_doc)
"""

with open("utils_process_docs.py", "w") as f:
    f.write(DOC_TO_TEXT)

!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography_function_prompt_2 \
    --limit 10 \
    --output output/demo_mmlu_high_school_geography_function_prompt_2/ \
    --log_samples

We hope that this explainer gives you a sense of what can be done with and how to work with LM-Evaluation-Harnes v0.4.0 ! 

For more information, check out our documentation pages in the `docs/` folder, and if you have questions, please raise them in GitHub issues, or in #lm-thunderdome or #release-discussion on the EleutherAI discord server.